### Init model from config.yaml (and checkpoint)

In [ ]:
import os
import yaml

import hydra
import torch
import transformers
from transformers import AutoModel
from composer.models import write_huggingface_pretrained_from_composer_checkpoint
from omegaconf import OmegaConf

In [ ]:
path_to_ckpt_dir = ""  # TODO: Fill in ckpt_dir path
with open(os.path.join(path_to_ckpt_dir, "config.yaml", "rb")) as f:
    config = yaml.safe_load(f)

In [ ]:
tokenizer = hydra.utils.instantiate(config.tokenizer)
model = hydra.utils.instantiate(
    config.model,
    _convert_="all",
)

In [ ]:
ckpt_file = "latest-rank0.pt"  # TODO: Choose ckpt file
try:
    ckpt = torch.load(
        os.path.join(path_to_ckpt_dir, "checkpoints", ckpt_file), weights_only=False
    )
    state_dict = ckpt["state"]["model"]
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(state_dict, "model.")
    model.load_state_dict(state_dict)
except FileNotFoundError as e:
    print(e)

### Test push to hub

In [ ]:
from src.utils import push_to_hub

In [ ]:
push_to_hub(
    model=model,
    tokenizer=tokenizer,
    # repo_id="",  # TODO: Set this to local path and `local=True` below to test local saving
    # local=True,  # TODO: Uncomment for testing local saving
)

#### Test round trip

In [ ]:
from transformers import AutoModel

In [ ]:
test_model = AutoModel.from_pretrained(
    "kuleshov-group/dllm-dev",  # TODO: Change to local path from above to test loading from locally saved checkpoint
    trust_remote_code=True,
)

In [ ]:
test_model.config

In [ ]:
test_model